ada app assessment

q3. In which year (2000 or 2015) did storms have a higher monetary impact within the boundaries of the 13 original colonies?

In [1]:
import pandas as pd
import numpy as np
storm2000 = pd.read_csv('StormEvents_details-ftp_v1.0_d2000_c20160223.csv')
storm2015 = pd.read_csv('StormEvents_details-ftp_v1.0_d2015_c20160810.csv')